In [1]:
import os, time

In [2]:
%run ./postgres_utilities.ipynb

In [3]:
embedding_endpoint =  os.environ.get('embedding_model_endpoint')
model_name = 'grit-llm'
number_data_rows = 1000

In [4]:
connect_db()

In [5]:
sql = """
    select aidb.delete_model('grit-llm');
    """

In [6]:
execute_sql(sql)

In [7]:
sql = f"""
    select aidb.create_model(
    'grit-llm',
    'embeddings',
    '{{"model":"grit-llm", "url":"{embedding_endpoint}", 
    "dimensions":4096}}'::JSONB,
    '{{"api_key":""}}'::JSONB);
    """

In [8]:
execute_sql(sql)

In [9]:
#Compute time for 1k embeddings

In [10]:
sql = f"select description from film limit {number_data_rows}"
results = execute_sql_results(sql)

In [11]:
encode_text_sql = "select aidb.encode_text('grit-llm', %s)"

In [12]:
test_description = "Starting online embedding test using edb and default vllm configuration"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Embeddings computed ...", end=" ")
for index, film in enumerate(results):
    description = film[0]
    embedding = execute_sql_results_params(encode_text_sql, description)
    assert len(embedding[0][0]) == 4096
    if index % 100 == 0:
        print(f"{index}", end=" ")
toc = time.perf_counter()
print(f"{index+1}")
print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
print(len(test_description) * "-")

Starting online embedding test using edb and default vllm configuration
-----------------------------------------------------------------------
Embeddings computed ... 0 100 200 300 400 500 600 700 800 900 1000
Time to compute 1000 embeddings: 61.8637 seconds
-----------------------------------------------------------------------


In [13]:
close_db()